In [ ]:
!gcloud auth application-default login

In [1]:
!pip3 install --upgrade "google-cloud-aiplatform>=1.38"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 38.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.67.1
    Uninstalling google-cloud-aiplatform-1.67.1:
      Successfully uninstalled google-cloud-aiplatform-1.67.1


In [1]:
!gcloud auth application-default set-quota-project "zeta-pivot-438217-s0"


Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "zeta-pivot-438217-s0" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [2]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="zeta-pivot-438217-s0", location="us-central1")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 256,
    "temperature": 0.2,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained("text-bison")
response = model.predict(
    """Summarize the following conversation between a service rep and a customer in a few sentences. Use only the information from the conversation.

Service Rep: How may I assist you today?
Customer: I need to change the shipping address for an order.
Service Rep: Ok, I can help you with that if the order has not been fulfilled from our warehouse yet. But if it has already shipped, then you will need to contact the shipping provider. Do you have the order ID?
Customer: Yes, it\'s 88986367.
Service Rep: One minute please while I pull up your order information.
Customer: No problem
Service Rep: Ok, it looks like your order was shipped from our warehouse 2 days ago. It is now in the hands of  the shipping provider, so you will need to contact them to update your delivery details. You can track your order with the shipping provider here: https://www.shippingprovider.com
Customer: Sigh, ok.
Service Rep: Is there anything else I can help you with today?
Customer: No, thanks.""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  A customer contacted customer service to change the shipping address for an order. The service rep informed them that the order had already been shipped and provided the customer with a link to track the order and contact the shipping provider to update the delivery details. The customer was disappointed but understood the situation.


In [3]:
response = model.predict(
    """Ticket: I lost my credit card numbered 12345. Can you help with deactivating the card?
Category: Credit card

Ticket: I would like to change the address associated with my account. I have been calling the bank multiple times but couldn\'t get through. Please help me.
Category: Bank account services

Ticket: good morning my name is xxxx xxxx and i appreciate it if you could help me put a stop to chase bank cardmember services. I wrote to chase asking for debt verification and what they sent me a statement which is not acceptable i am asking the bank to validate the debt instead i been receiving mail every month from them attempting to collect a debt i have a right to know this information as a consumer chase account xxxx xxxx xxxx xxxx thanks in advance for your help
Category: Loans and Mortgages

Ticket: Customer: Hello, I\'m calling to ask about your credit card rewards programs.\\nBank: Hi, thank you for calling [bank name]. Can I have your name and customer ID, please?\\nCustomer: Sure, my name is [customer name] and my customer ID is 1312345681.\\nBank: Thank you, Mr./Ms. [customer name]. We offer a variety of credit card rewards programs to meet your needs. Would you like me to tell you more about them?
Category:
""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  Credit card


In [5]:
import pandas as pd

banking_df = pd.read_csv("/content/banking_dataset.csv", encoding="ISO-8859-1")
banking_df.head()

,"ÿ""customer_id""",call_date,call_time,call_transcript
0,1234567890,2023-10-31,13:55:00,"Customer: Hello, I'm calling to check on the s..."
1,3456789012,2023-10-31,14:05:00,"Customer: Hello, I'm calling to inquire about ..."
2,1098765432,2023-10-31,14:10:00,"Customer: Hello, I'm calling to transfer money..."
3,2134567890,2023-10-31,14:15:00,"Customer: Hello, I'm calling to dispute a char..."
4,4567890123,2023-10-31,14:20:00,"Customer: Hello, I'm calling to ask about your..."


In [6]:
banking_df['Classification'] = banking_df.apply(lambda x:model.predict(
    """Multi-choice problem: Define the category of the ticket?
Categories:
- Credit card
- Bank account services
- Loans and Mortgages""" + 'Ticket: ' + x['call_transcript'] + "Category: """).text, axis=1)

In [7]:
banking_df.head()

,"ÿ""customer_id""",call_date,call_time,call_transcript,Classification
0,1234567890,2023-10-31,13:55:00,"Customer: Hello, I'm calling to check on the s...",Loans and Mortgages
1,3456789012,2023-10-31,14:05:00,"Customer: Hello, I'm calling to inquire about ...",Investment products
2,1098765432,2023-10-31,14:10:00,"Customer: Hello, I'm calling to transfer money...",Bank account services
3,2134567890,2023-10-31,14:15:00,"Customer: Hello, I'm calling to dispute a char...",Credit card
4,4567890123,2023-10-31,14:20:00,"Customer: Hello, I'm calling to ask about your...",Loans and Mortgages


In [8]:
banking_df

,"ÿ""customer_id""",call_date,call_time,call_transcript,Classification
0,1234567890,2023-10-31,13:55:00,"Customer: Hello, I'm calling to check on the s...",Loans and Mortgages
1,3456789012,2023-10-31,14:05:00,"Customer: Hello, I'm calling to inquire about ...",Investment products
2,1098765432,2023-10-31,14:10:00,"Customer: Hello, I'm calling to transfer money...",Bank account services
3,2134567890,2023-10-31,14:15:00,"Customer: Hello, I'm calling to dispute a char...",Credit card
4,4567890123,2023-10-31,14:20:00,"Customer: Hello, I'm calling to ask about your...",Loans and Mortgages
5,5678901234,2023-10-31,14:25:00,"Customer: Hello, I'm calling to open a busines...",Bank account services
6,6789012345,2023-10-31,14:30:00,"Customer: Hello, I'm calling to ask about your...",Bank account services
7,7890123456,2023-10-31,14:35:00,"Customer: Hello, I'm calling to ask about your...",Loans and Mortgages
8,8901234567,2023-10-31,14:40:00,"Customer: Hello, I'm calling to report a lost ...",Bank account services
9,9012345678,2023-10-31,14:45:00,"Customer: Hello, I'm calling to inquire about ...",Loans and Mortgages


In [9]:
import vertexai
from vertexai.language_models import ChatModel, InputOutputTextPair
from vertexai import generative_models

vertexai.init(project="zeta-pivot-438217-s0", location="us-central1")
chat_model = ChatModel.from_pretrained("chat-bison")
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 1
}
safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }
chat = chat_model.start_chat(
    context="""""",
)
response = chat.send_message("""Hello. Please help extract the main issues from this customer\\\'s complaint:

I like to start off by saying my family loves your restaurant...I remember the days when our kids had birthday parties there. But the decline in service is scary and truly hurting the reputation. Our family went to your location on Jessup Rd in Summerville, SC, the other day. We drove up to the speaker and placed our order. Drove around and there was one car. That car was there a while then they pulled around to the other side. We moved up and waited 28 minutes, and watched about 4 other cars leave. In those 28 minutes, not one employee came to the window. We watched as they were talking and laughing with each other going back and forth in the lobby. We noticed that the people who were in front of us ended up going inside. We finally drove off and went to another restaurant. Fast easy and friendly service. I truly hope you guys can save BK. But I have to say I\\\'m so disappointed as a supporter.""", **parameters)
print(f"Response from Model: {response.text}")
response = chat.send_message("""How can we revise our practices or protocols to prevent future incidents like these?""", **parameters)
print(f"Response from Model: {response.text}")
response = chat.send_message("""tell me more about your insurance property""", **parameters)
print(f"Response from Model: {response.text}")

Response from Model:  Certainly! Here are the main issues based on the customer's complaint:

- **Slow service:** The customer waited for 28 minutes at the drive-thru without being served, while observing employees socializing inside the restaurant.

- **Lack of attention to customers:** No employees came to the window to acknowledge or assist the customer during the long wait.

- **Poor customer service:** The customer noticed that other cars left the restaurant without being served, and those who went inside seemed to receive faster service. This suggests a lack of attention to drive-thru customers.

- **Loss of reputation:** The customer expresses concern about the decline in service and its impact on the restaurant's reputation.

- **Disappointment as a supporter:** The customer expresses disappointment as a long-time supporter of the restaurant, highlighting the gap between their positive past experiences and the current poor service.
Response from Model:  Here are some strategies